In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [4]:
# Generate synthetic input features
n_samples = 1000
df = pd.DataFrame({
    'CDT': np.random.normal(300, 50, n_samples),
    'NHT.': np.random.uniform(0, 24, n_samples),
    'NMT.': np.random.normal(15, 5, n_samples),
    'Tmin.': np.random.normal(10, 5, n_samples),
    'NHR0.5': np.random.uniform(0, 12, n_samples),
    'Max.Rain': np.random.exponential(10, n_samples),
    'DMT.': np.random.normal(20, 5, n_samples),
    'NHRH90.': np.random.uniform(0, 24, n_samples)
})


In [5]:
# Generate synthetic target (disease incidence)
weights = np.array([0.3, 0.2, 0.1, 0.05, 0.15, 0.1, 0.05, 0.05])
df['Disease_Incidence'] = (df.values @ weights + np.random.normal(0, 5, n_samples)).clip(0, 100)

In [6]:
# Split data
X = df.drop(columns='Disease_Incidence')
y = df['Disease_Incidence']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:
# Normalize features
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [8]:
# Build ANN model
model = Sequential([
    Dense(16, input_dim=8, activation='relu'),
    Dense(8, activation='relu'),
    Dense(1, activation='linear')
])

/Users/mohammad.aval/miniconda3/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
# Compile using gradient descent
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
              loss='mean_squared_error',
              metrics=['mae'])

In [10]:
# Train the model
model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.2)                       # Output layer

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 1250098.8750 - mae: 469.6875 - val_loss: 15640.5098 - val_mae: 124.6352
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 13221.7041 - mae: 114.1440 - val_loss: 6970.7056 - val_mae: 82.8501
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 5978.7935 - mae: 76.4335 - val_loss: 3127.7964 - val_mae: 54.9656
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2705.4260 - mae: 50.9502 - val_loss: 1427.3606 - val_mae: 36.3478
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1246.6058 - mae: 33.8634 - val_loss: 676.8910 - val_mae: 24.3283
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 616.2635 - mae: 23.3580 - val_loss: 349.8669 - val_mae: 17.0302
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 320.4564 - mae: 16.3881 - val_loss: 207.7705 - val_mae: 12.9693
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 194.9603 - mae: 12.5558 - val_loss: 146.9518 - val_mae: 10.727

In [11]:
# Evaluate
loss, mae = model.evaluate(X_test_scaled, y_test)
print(f"Test Loss: {loss:.2f}, Test MAE: {mae:.2f}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 123.8524 - mae: 8.5240  
Test Loss: 112.00, Test MAE: 8.31
